# 01-download_dataset_audio

This notebook downloads the audio recordings listed in the `annotation.json` from [Dunya makam music corpus](https://dunya.compmusic.upf.edu/).  The annotations are saved in a `mlflow` run called `download_audio` under an experiment named `data_processing`.

In [1]:
import mre


## Stop if audio were fetched in the past

Downloading all recordings may take a long time. It is better to check if there is already a run in advance.

In [2]:
run = mre.mlflow_common.get_run_by_name(
    mre.data.Audio.EXPERIMENT_NAME, mre.data.Audio.RUN_NAME)

if run is not None:
    raise ValueError(
        "There is already a run for %s:%s. Overwriting is not "
        "permitted. Please delete the run manually if you want "
        "to log the annotations again."
        % (mre.data.Audio.RUN_NAME, run.run_id))


No runs with the name audio in experiment data_processing


# Read annotations from mlflow


In [3]:
annotations = mre.data.Annotation()
annotations.from_mlflow()
display(annotations.head())

,mbid,verified,tonic,makam,observations,dunya_uid,mb_url
0,00f1c6d9-c8ee-45e3-a06f-0882ebcb4e2f,False,256.0,Acemasiran,,00f1c6d9-c8ee-45e3-a06f-0882ebcb4e2f,http://musicbrainz.org/recording/00f1c6d9-c8ee...
1,168f7c75-84fb-4316-99d7-acabadd3b2e6,False,115.2,Acemasiran,,168f7c75-84fb-4316-99d7-acabadd3b2e6,http://musicbrainz.org/recording/168f7c75-84fb...
2,24f549dd-3fa4-4e9b-a356-778fbbfd5cad,False,232.5,Acemasiran,,24f549dd-3fa4-4e9b-a356-778fbbfd5cad,http://musicbrainz.org/recording/24f549dd-3fa4...
3,407bb0b4-f19b-42ab-8c0a-9f1263126951,False,233.5,Acemasiran,,407bb0b4-f19b-42ab-8c0a-9f1263126951,http://musicbrainz.org/recording/407bb0b4-f19b...
4,443819eb-6092-420c-bd86-d946a0ad6555,False,219.6,Acemasiran,,443819eb-6092-420c-bd86-d946a0ad6555,http://musicbrainz.org/recording/443819eb-6092...


# Download audio recordings from Dunya and log to mlflow


In [4]:
audio = mre.data.Audio()
audio.from_dunya(annotations.data)
audio.log()


100%|██████████| 1000/1000 [41:42<00:00,  2.50s/it] 
No runs with the name audio in experiment data_processing
